In [32]:
import json
import os
import pandas as pd
import pdb
import traceback
import ast
# This file comes right after using gpt 3.5 turbo to extract the attrbutes from the origional string
with open('/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/extracted_attributes_1225_1425.json', 'r') as file:
    data = json.load(file)
df = pd.DataFrame.from_dict(data, orient='index')

In [33]:
df.iloc[0]['extracted_attributes']

'{\n  "Ratings for Response A": {\n    "Hallucinations": 0.3,\n    "Helpfulness": 0.7,\n    "Quality": 0.6,\n    "Spatial-Awareness": 0.7,\n    "Domain-Knowledge": 0.5\n  },\n  "Ratings for Response B": {\n    "Hallucinations": 0.7,\n    "Helpfulness": 0.6,\n    "Quality": 0.5,\n    "Spatial-Awareness": 0.5,\n    "Domain-Knowledge": 0.5\n  }\n}'

In [34]:
df2 = pd.read_csv('/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/temp_30_1224.csv')
# check if two df have the same columns
df2.iloc[0]['extracted_attributes']

'{\n  "Ratings for Response A": {\n    "Hallucinations": 0.1,\n    "Helpfulness": 0.8,\n    "Quality": 0.8,\n    "Spatial-Awareness": 1.0,\n    "Domain-Knowledge": 0.8\n  },\n  "Ratings for Response B": {\n    "Hallucinations": 0.1,\n    "Helpfulness": 0.8,\n    "Quality": 0.8,\n    "Spatial-Awareness": 1.0,\n    "Domain-Knowledge": 0.7\n  }\n}'

In [35]:
with open('/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/extracted_attributes_1525_2125.json', 'r') as file:
    data = json.load(file)
df3 = pd.DataFrame.from_dict(data, orient='index')

In [38]:
import pandas as pd

# # Paths to the JSON files
# file1 = 'extracted_attributes_sft_0_1224-likert_scale1118.json'
# file2 = 'extracted_attributes_sft_1225-1525-likert_scale1118.json'
# file3 = 'extracted_attributes_sft_30_1525-2125-likert_scale1118.json'

# # Load each file into a DataFrame
# df1 = pd.read_json(file1, orient='records')
# df2 = pd.read_json(file2, orient='records')
# df3 = pd.read_json(file3, orient='records')

# Concatenate the DataFrames
combined_df = pd.concat([df, df2, df3])

# Save the combined DataFrame as a new JSON file
# combined_df.to_json('combined.json', orient='records')

In [42]:
def _trim_conversation(conversation):
    # Parse the conversation string into a list of dictionaries
    if isinstance(conversation, str):
        conversation_list = ast.literal_eval(conversation)
    elif isinstance(conversation, list):
        conversation_list = conversation
    else:
        raise Exception("The conversation is not in the correct format.")
    # Keep only the last two turns of the conversation (one human and one gpt turn)
    if len(conversation_list) >= 2:
        trimmed_conversation = conversation_list[-2:]
    else:
        raise Exception("The conversation is too short to trim.")
    # return only the human turn
    trimmed_conversation[-2]['value'] = '<image>\n' + trimmed_conversation[-2]['value']
    return trimmed_conversation[-2]['value']

def _get_output(conversation):
    # Parse the conversation string into a list of dictionaries
    if isinstance(conversation, str):
        conversation_list = ast.literal_eval(conversation)
    elif isinstance(conversation, dict):
        conversation_list = conversation
    else:
        raise Exception("The conversation is not in the correct format.")
    # Keep only the last two turns of the conversation (one human and one gpt turn)

    # return only the human turn
    output = conversation_list['value']
    return output

def _get_gpt4_response(gpt4v_response):
    # Parse the conversation string into a list of dictionaries
    if isinstance(gpt4v_response, str):
        gpt4v_response = ast.literal_eval(gpt4v_response)
    elif isinstance(gpt4v_response, dict):
        gpt4v_response = gpt4v_response
    else:
        raise Exception("The conversation is not in the correct format.")
    
    return gpt4v_response['choices'][0]['message']['content']


def extract_data(df):
    extracted_data = []
    for index, row in df.iterrows():
        try:
            # Extract the desired information from each row
            human_turn = _trim_conversation(row['conversations'])
            output_1 = _get_output(row['output_1'])
            output_2 = _get_output(row['output_2'])
            gpt_response_text = _get_gpt4_response(row['gpt4v_response'])
        except Exception as e:
            # Handle the case when an exception occurs
            print(f"Error occurred for row {index}: {e}")
            human_turn = ""
            output_1 = ""
            output_2 = ""
            gpt_response_text = ""
        
        # Append the extracted data to the list
        extracted_data.append((human_turn, output_1, output_2, gpt_response_text))
    
    # Create new columns for the extracted data
    df['question_string'] = [data[0] for data in extracted_data]
    df['Output_1_string'] = [data[1] for data in extracted_data]
    df['Output_2_string'] = [data[2] for data in extracted_data]
    df['GPT_teacher_critiq_string'] = [data[3] for data in extracted_data]
    
    return df

combined_extract = extract_data(combined_df)
combined_extract['question_id'] = combined_extract.apply(lambda row: row['id'], axis=1)
combined_extract.to_json('/home/ubuntu/RLHF/LLaVA-RLHF/RLHF/gpt4_ratings/training_data/teacher-critique-0-2125.json', orient='records', lines=True)


In [29]:
df2_extract['GPT_teacher_critiq_string'][0]

'Response A and B are both based on the same image which depicts a cow in a field, separated from the viewer by a barbed wire fence. Both responses assume the cow\'s living conditions to be good based on its environment, which is a spacious area with grass coverage.\n\nResponse A goes into more detail about the conditions necessary for cow welfare, suggesting a holistic approach to understanding the image. It correctly identifies a single cow and refers to the fence that delineates the cow\'s grazing area. Additionally, it gives an extensive interpretation of how the setting might affect the well-being of the cow.\n\nResponse B similarly identifies the cow and its environment, mentioning the barbed wire fence and implying that the cow is well-taken care of due to its environment. It portrays the cow as healthy-looking and states the natural habitat is suitable for its needs.\n\nHere are the ratings for each response:\n\nRatings for Response A:\n```json\n{\n  "Hallucinations": 0.1, // T

In [ ]:
# get the last question turn as prompt from 'conversations'
human_turn = _trim_conversation(df2.iloc[0]['conversations'])
# get the first output_1 
output_1 = _get_output(df2.iloc[0]['output_1'])
# get the second output_2
output_2 = _get_output(df2.iloc[0]['output_2'])
# provide the gpt4v_response
gpt_repsonse_text = _get_gpt4_response(df2.iloc[0]['gpt4v_response'])


In [15]:
len(df2)

1151